# Anwendung und Vergleich von Machine Learning und Deep Learning Algorithmen zur Vorhersage von COVID-19 Kennzahlen

## Regression
Mit einer Regression werden mit unabhängigen Variabeln $x^{n}$ abhängige Variabeln $y^{n}$ beschrieben. Die unabhängigen Variabeln sind numerisch. 
Die lineare Regression ist die einfachste Art der Regression. Sie geht davon aus dass die Beziehung linear ist. 
Bei der Polynomregression werden die Polynome von x verwendet: ${x}$, $x^{2}$, $x^{3}$...

## Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

import plotly.graph_objs as go
from ipywidgets import interact

%matplotlib inline

## Laden der Daten

## Modelle

### Lineare Regression

In [ ]:
lr = LinearRegression(normalize=True)
scores = cross_val_score(lr, X=X, y=y, cv=10, scoring='neg_mean_absolute_error')*-1
scores

### Polynomiale Regression

In [ ]:
rr = Ridge(alpha=0.1, normalize=True, random_state=31337)

for i in [5,6,7]:
    poly = PolynomialFeatures(i)
    X = df[feature_columns].values
    X = poly.fit_transform(X)
    scores = cross_val_score(rr, X=X, y=y, cv=10, scoring='neg_mean_absolute_error')*-1
    print("With polynomial features and interactions of degree {}, we get an error of {:.2f}".format(i, np.mean(scores)))

## Training der Modelle

## Evaluation